In [ ]:
import cobra
import d3flux as d3f

# Plotting utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks', color_codes=True)
%matplotlib inline

## Elementary flux modes

<img src='figs/em_decomp_yield_1.svg' width='500', style='display: block; margin: 0 auto;'>

Decomposes the network into a unique set of sub-graphs

$$
\begin{aligned}
\mathbf{S}\mathbf{r}_i &= 0\\
\end{aligned}
$$

Any feasible flux can be expressed by a nonnegative combination of EFMs

$$
\mathbf{v} = a_0 \mathbf{r}_0 + a_1 \mathbf{r}_1 + \cdots
$$

EFMS reveal the feasible flux space of the model

<img src='figs/em_decomp_yield_surf.svg' width='300', style='display: block; margin: 0 auto;'>


### Elementary modes and mimimum cut sets

Wrapper code for `efmtool` and `MhsCalculator` are available at [<i class='fa fa-github'></i> pstjohn/pyefm](https://github.com/pstjohn/pyefm)

In [ ]:
from pyefm import calculate_elementary_modes, calculate_minimum_cut_sets

In [ ]:
from cobra.core import Metabolite, Reaction, Model

simple_model = Model('simple_model')

A = Metabolite('A')
B = Metabolite('B')
C = Metabolite('C')
D = Metabolite('D')
E = Metabolite('E')
P = Metabolite('P')

R1 = Reaction('R1')
R2 = Reaction('R2')
R3 = Reaction('R3')
R4 = Reaction('R4')
R5 = Reaction('R5')
R6 = Reaction('R6')
R7 = Reaction('R7')
R8 = Reaction('R8')
R9 = Reaction('R9')
R10 = Reaction('R10')

simple_model.add_metabolites([A, B, C, D, E, P])
simple_model.add_reactions([R1, R2, R3, R4, R5, R6, R7, R8, R9, R10])

simple_model.reactions.R1.build_reaction_from_string('--> A')
simple_model.reactions.R2.build_reaction_from_string('<--> B')
simple_model.reactions.R3.build_reaction_from_string('P -->')
simple_model.reactions.R4.build_reaction_from_string('E -->')
simple_model.reactions.R5.build_reaction_from_string('A --> B')
simple_model.reactions.R6.build_reaction_from_string('A --> C')
simple_model.reactions.R7.build_reaction_from_string('A --> D')
simple_model.reactions.R8.build_reaction_from_string('B <--> C')
simple_model.reactions.R9.build_reaction_from_string('B --> P')
simple_model.reactions.R10.build_reaction_from_string('C + D --> E + P')

d3f.flux_map(simple_model,
             display_name_format=lambda x: x.id,
             flux_dict={r.id: None for r in simple_model.reactions},
             figsize=(1000, 300))

In [ ]:
simple_efms = calculate_elementary_modes(simple_model)
simple_efms

## Minimum cut sets

Minimum cut sets find the smallest sets of reactions to remove all target EFMs. *Constrained* minimum cut sets remove all target modes while keeping at least one desired mode

* Hadicke O, Klamt S. Computing complex metabolic intervention strategies using constrained minimal cut sets. Metab Eng. 2011;13(2):204-213. [doi:10.1016/j.ymben.2010.12.004.](https://www.ncbi.nlm.nih.gov/pubmed/21147248)

Make P without making E?

In [ ]:
good = simple_efms[simple_efms.R3 > 0]
bad = simple_efms[simple_efms.R4 > 0]

mcs = calculate_minimum_cut_sets(good, bad, keep=1, verbose=False)

# Some post-processing code
sorted_cutsets = mcs.iloc[mcs.sum(1).argsort()]
rxn_kos = sorted_cutsets.T.apply(
    lambda x: [simple_model.reactions.get_by_id(xi).reaction
               for xi in x.loc[x].index])
rxn_kos

### Muconate production in *P. putida*

In [ ]:
from Models.putida import v9_presentation

model = v9_presentation()
model.optimize()

d3f.flux_map(model)

This calculation takes a couple minutes, so I've cached the results from ahead of time.
The actual calculation is just done with
```python
efms = calculate_elementary_modes(model)
efms.to_pickle('data/muconate_efms.p')
```

In [ ]:
efms = pd.read_pickle('data/muconate_efms.p')

# Normalize by glucose uptake
efms = efms.divide(-efms.EX_glc_e, 0)
efms.shape

In [ ]:
good    = efms[(efms.muconate_sink > 0.3) & (efms.Biomass_Ecoli_core_w_GAM > .01)]
bad     = efms[(efms.muconate_sink < 0.3) & (efms.Biomass_Ecoli_core_w_GAM > .0025)]
neutral = efms[(~efms.index.isin(good.index) & ~efms.index.isin(bad.index))]

In [ ]:
import matplotlib

def plot_efms(ax, efms, color=None, patch=None):
    ax.plot(efms.muconate_sink, efms.Biomass_Ecoli_core_w_GAM, '.', color=color, rasterized=True)

def lighten_color(color, degree):
    cin = matplotlib.colors.colorConverter.to_rgb(color)
    cw = np.array([1.0]*3)
    return tuple(cin + (cw - cin)*degree)

In [ ]:
with sns.axes_style('ticks'):
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)

    plot_efms(ax, neutral, 'b')
    plot_efms(ax, good, 'g', patch='Good')
    plot_efms(ax, bad, 'r', patch='Bad')
    
    ax.add_patch(plt.Polygon(np.array([[0, .0025], [0, .09], [.3, .09], [.3, .0025]]),
                 facecolor=lighten_color('r', .9), edgecolor='none'))
    ax.add_patch(plt.Polygon(np.array([[0.3, .01], [0.3, .06], [.7, .06], [.7, .01]]),
                 facecolor=lighten_color('g', .9), edgecolor='none'))

    sns.despine()
    
    ax.text(.175, .09, 'EFMs\nto remove',
            fontdict={'color' : 'r', 'weight' : 'bold', 'size' : 18, 'style' : 'italic'})
    ax.text(.55, .04, 'EFMs\nto keep',
            fontdict={'color' : 'g', 'weight' : 'bold', 'size' : 18, 'style' : 'italic'})
    
    ax.set_xlabel('Muconate yield (mol/mol)')
    ax.set_ylabel('Biomass yield (g DCW/mol)')
    
    ax.set_ylim([0, .09])
    ax.set_xlim([0, 0.8])
    sns.despine(trim=True, offset=10)
    fig.tight_layout()

In [ ]:
essential_reactions = model.reactions.query(lambda x: x != set('c'), attribute='compartments')
essential_reactions += [model.reactions.HCO3E]
cutsets = calculate_minimum_cut_sets(good, bad, keep=1,
                                     essential_reactions=essential_reactions,
                                     verbose=False)

In [ ]:
sorted_cutsets = cutsets.iloc[cutsets.sum(1).argsort()]
rxn_kos = sorted_cutsets.T.apply(lambda x: list(x.loc[x].index))
rxn_kos.name = 'rxnkos'
rxn_kos.head(10)

In [ ]:
with model:
    for rxn in rxn_kos.iloc[1]:
        model.reactions.get_by_id(rxn).knock_out()
    model.summary()
    svg = d3f.flux_map(model, inactive_alpha=0.5)

In [ ]:
svg